In [1]:
import pandas as pd
from datetime import datetime
from rich import print
from rich.pretty import Pretty

pd.set_option('display.max_columns', None)

In [2]:
df_work_raw = catalog.load('raw/openalex/work_dev#parquet')

[08/26/25 13:47:00] INFO     Loading data from raw/openalex/work_dev#parquet                    ]8;id=137852;file:///root/kedro-unlp/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=403793;file:///root/kedro-unlp/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (ParquetDataset)...                                                                   

**Estructura de Authorships**

In [3]:
from rich import print, pretty

first_authorship = df_work_raw['authorships'].iloc[0][0]
print(pretty.Pretty(first_authorship, expand_all=False))

{
    'affiliations': array([{'institution_ids': array(['https://openalex.org/I392282'], dtype=object), 
'raw_affiliation_string': 'University at Albany, 1400 Washington Ave, Albany, NY 12222, United States of 
America'}],
      dtype=object),
    'author': {
        'display_name': 'the Atlas Collaboration',
        'id': 'https://openalex.org/A5107845465',
        'orcid': None
    },
    'author_position': 'first',
    'countries': array(['US'], dtype=object),
    'institutions': array([{'country_code': 'US', 'display_name': 'University at Albany, State University of New 
York', 'id': 'https://openalex.org/I392282', 'lineage': array(['https://openalex.org/I392282'], dtype=object), 
'ror': 'https://ror.org/012zs8222', 'type': 'funder'}],
      dtype=object),
    'is_corresponding': False,
    'raw_affiliation_strings': array(['University at Albany, 1400 Washington Ave, Albany, NY 12222, United States 
of America'],
      dtype=object),
    'raw_author_name': 'The ATLAS Collaboration'
}

# Transformaciones

In [4]:
# Seleccionar las columnas necesarias y convertir los tipos de datos
df_work2authorships = df_work_raw[['id', 'authorships']].convert_dtypes()
df_work2authorships.rename(columns={"id": "work_id"}, inplace=True)

In [5]:
df_work2authorships.head(3)

,work_id,authorships
0,https://openalex.org/W4211046286,[{'affiliations': [{'institution_ids': array([...
1,https://openalex.org/W4292403095,[{'affiliations': [{'institution_ids': array([...
2,https://openalex.org/W2501864044,[{'affiliations': [{'institution_ids': array([...


In [6]:
# Expandir la lista de authorships
df_work2authorships_exploded = df_work2authorships.explode('authorships', ignore_index=True)

In [7]:
df_work2authorships_exploded.head(3)

,work_id,authorships
0,https://openalex.org/W4211046286,{'affiliations': [{'institution_ids': ['https:...
1,https://openalex.org/W4211046286,{'affiliations': [{'institution_ids': ['https:...
2,https://openalex.org/W4211046286,{'affiliations': [{'institution_ids': ['https:...


In [8]:
# Normalizar la información de authorships
df_authorships_norm = pd.json_normalize(df_work2authorships_exploded['authorships'])
df_authorships_norm.rename(columns={"author.id": "author_id"}, inplace=True)

In [9]:
df_authorships_norm.head(3)

,affiliations,author_position,countries,institutions,is_corresponding,raw_affiliation_strings,raw_author_name,author.display_name,author_id,author.orcid
0,[{'institution_ids': ['https://openalex.org/I3...,first,[US],"[{'country_code': 'US', 'display_name': 'Unive...",False,"[University at Albany, 1400 Washington Ave, Al...",The ATLAS Collaboration,the Atlas Collaboration,https://openalex.org/A5107845465,None
1,[{'institution_ids': ['https://openalex.org/I2...,middle,[FR],"[{'country_code': 'FR', 'display_name': 'Aix-M...",False,"[CPPM, Aix-Marseille Université, CNRS/IN2P3, M...",G Aad,G. Aad,https://openalex.org/A5100769897,https://orcid.org/0000-0002-6665-4934
2,[{'institution_ids': ['https://openalex.org/I4...,middle,[TR],"[{'country_code': 'TR', 'display_name': 'Boğaz...",False,"[Faculty of Sciences, Department of Physics, B...",E Abat,E. Abat,https://openalex.org/A5086185112,None


In [10]:
# Combinar work_id con la información normalizada de authorships
df_work2authorships = df_work2authorships_exploded[['work_id']].join(df_authorships_norm)

In [11]:
df_work2authorships.head(3)

,work_id,affiliations,author_position,countries,institutions,is_corresponding,raw_affiliation_strings,raw_author_name,author.display_name,author_id,author.orcid
0,https://openalex.org/W4211046286,[{'institution_ids': ['https://openalex.org/I3...,first,[US],"[{'country_code': 'US', 'display_name': 'Unive...",False,"[University at Albany, 1400 Washington Ave, Al...",The ATLAS Collaboration,the Atlas Collaboration,https://openalex.org/A5107845465,None
1,https://openalex.org/W4211046286,[{'institution_ids': ['https://openalex.org/I2...,middle,[FR],"[{'country_code': 'FR', 'display_name': 'Aix-M...",False,"[CPPM, Aix-Marseille Université, CNRS/IN2P3, M...",G Aad,G. Aad,https://openalex.org/A5100769897,https://orcid.org/0000-0002-6665-4934
2,https://openalex.org/W4211046286,[{'institution_ids': ['https://openalex.org/I4...,middle,[TR],"[{'country_code': 'TR', 'display_name': 'Boğaz...",False,"[Faculty of Sciences, Department of Physics, B...",E Abat,E. Abat,https://openalex.org/A5086185112,None


In [ ]:
# Extraer la relación work-author
df_work2author = df_work2authorships[['work_id', 'author_id', 'author_position']]

In [ ]:
df_work2author.head(3)

In [ ]:
# Expandir la lista de instituciones asociadas a cada autor
df_work2institution_exploded = df_work2authorships.explode('institutions', ignore_index=True)

In [ ]:
df_work2institution_exploded.head(3)

In [ ]:
# Normalizar la información de instituciones
df_institution_norm = pd.json_normalize(df_work2institution_exploded['institutions'])
df_institution_norm.drop(columns=['lineage'], errors='ignore', inplace=True)

In [ ]:
df_institution_norm.head(3)

In [ ]:
# Combinar author_id con la información normalizada de instituciones
df_author2institution = df_work2institution_exploded[['author_id']].join(df_institution_norm)

In [ ]:
df_author2institution.head(3)

In [ ]:
# Combinar work_id con la información normalizada de instituciones
df_work2institution = df_work2institution_exploded[['work_id']].join(df_institution_norm)

In [ ]:
df_work2institution.head(3)

# Nodo

In [ ]:
def land_openalex_work_authorships(df_work_raw):

    # Seleccionar las columnas necesarias y convertir los tipos de datos
    df_work2authorships = df_work_raw[['id', 'authorships']].convert_dtypes()
    df_work2authorships.rename(columns={"id": "work_id"}, inplace=True)

    # Expandir la lista de authorships
    df_work2authorships_exploded = df_work2authorships.explode('authorships', ignore_index=True)

    # Normalizar la información de authorships
    df_authorships_norm = pd.json_normalize(df_work2authorships_exploded['authorships'])
    df_authorships_norm.rename(columns={"author.id": "author_id"}, inplace=True)
    
    # Combinar work_id con la información normalizada de authorships
    df_work2authorships = df_work2authorships_exploded[['work_id']].join(df_authorships_norm)

    # Extraer la relación work-author
    df_work2author = df_work2authorships[['work_id', 'author_id', 'author_position']]

    # Expandir la lista de instituciones asociadas a cada autor
    df_work2institution_exploded = df_work2authorships.explode('institutions', ignore_index=True)

    # Normalizar la información de instituciones
    df_institution_norm = pd.json_normalize(df_work2institution_exploded['institutions'])
    df_institution_norm.drop(columns=['lineage'], errors='ignore', inplace=True)

    # Combinar author_id con la información normalizada de instituciones
    df_author2institution = df_work2institution_exploded[['author_id']].join(df_institution_norm)

    # Combinar work_id con la información normalizada de instituciones
    df_work2institution = df_work2institution_exploded[['work_id']].join(df_institution_norm)
    
    df_work2author['load_datetime'] = datetime.today()
    df_work2institution['load_datetime'] = datetime.today()
    df_author2institution['load_datetime'] = datetime.today()

    return df_work2author, df_work2institution, df_author2institution


## Ejecuto Nodo

In [ ]:
df_work2author, df_work2institution, df_author2institution = land_openalex_work_authorships(df_work_raw)

# Resultados

In [ ]:
df_work2author

In [ ]:
df_work2institution

In [ ]:
df_author2institution